In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
racelist_url = 'https://jra.jp/keiba/overseas/racelist/'
years = ['2016', '2017', '2018', '2019', '2020', '2021']

races = []
for year in years:
    yearly_racelist_url = racelist_url + year + '.html'
    html = requests.get(yearly_racelist_url)
    soup = BeautifulSoup(html.content, "html.parser")
    
    table = soup.findAll("tbody")[0]
    rows = table.findAll("tr")
    for row in rows:
        csvRow = []
        for cell in row.findAll('a'):
            if '/keiba/overseas/race/' in cell.get('href'):
                tx = cell.get('href')
                tx2 = tx.strip()
                csvRow.append(tx2)
        if len(csvRow) > 0:
            races.append(csvRow[0])


In [3]:
races

['/keiba/overseas/race/2016arc/index.html',
 '/keiba/overseas/race/2016melbourne/index.html',
 '/keiba/overseas/race/2016bc/index.html',
 '/keiba/overseas/race/2016hkir/vase/index.html',
 '/keiba/overseas/race/2016hkir/sprint/index.html',
 '/keiba/overseas/race/2016hkir/mile/index.html',
 '/keiba/overseas/race/2016hkir/cup/index.html',
 '/keiba/overseas/race/2017dubai/turf/index.html',
 '/keiba/overseas/race/2017dubai/sc/index.html',
 '/keiba/overseas/race/2017dubai/wc/index.html',
 '/keiba/overseas/race/2017qe2/index.html',
 '/keiba/overseas/race/2017belmont/index.html',
 '/keiba/overseas/race/2017arc/index.html',
 '/keiba/overseas/race/2017hkir/vase/index.html',
 '/keiba/overseas/race/2017hkir/sprint/index.html',
 '/keiba/overseas/race/2017hkir/mile/index.html',
 '/keiba/overseas/race/2017hkir/cup/index.html',
 '/keiba/overseas/race/2018dubai/gs/index.html',
 '/keiba/overseas/race/2018dubai/turf/index.html',
 '/keiba/overseas/race/2018dubai/sc/index.html',
 '/keiba/overseas/race/2018

In [4]:
analysis_table = pd.DataFrame()

for race in races:
    slash_indices = [n for n, v in enumerate(race) if v == '/']
    year_start = race.find('2')
    race_year = int(race[year_start:year_start+4])
    race_name = race[slash_indices[-3]+1:slash_indices[-2]] + '_' + race[slash_indices[-2]+1:slash_indices[-1]]
    race_name = race[year_start:slash_indices[-1]].replace('/', '_')
    print(race_name)

    if race_year >= 2021 or (race_year == 2020 and 'hkir' in race_name):
        result = pd.read_csv('data/' + race_name + '_tbody.csv', names=['place', 'no', 'horse', 'gender_age', 'weight', 'jockey', 'time', 'horse_weight', 'trainer', 'popularity'], )
        vote = pd.read_csv('data/' + race_name + '_win.csv', names=['no', 'odds', 'popularity'])
    else:
        result = pd.read_csv('data/' + race_name + '_table.csv', header=0, names=['place', 'no', 'horse', 'gender_age', 'jockey_weight', 'jockey', 'time', 'horse_weight', 'trainer', 'popularity'])
        vote = pd.read_csv('data/' + race_name + '_harai.csv', header=None, names=['type', 'no', 'odds', 'popularity', 'type_2', 'no_2', 'odds_2', 'popularity_2', 'type_3', 'no_3', 'odds_3', 'popularity_3'])

    result.dropna(subset=['popularity'], inplace=True)
    result['no'] = result['no'].apply(lambda x: int(x[:x.find('(')]))
    result['horse_nationality'] = result['horse'].apply(lambda x: x[x.find('(')+1: len(x)-1])
    vote['odds'] = vote['odds'].apply(lambda x: int(x.replace('円', '').replace(',', '')))
    vote['popularity'] = vote['popularity'].apply(lambda x: int(x.replace('番人気', '')))

    for i in range(1,4):
        if result[result['popularity'] == i]['no'].values[0] == vote['no'].values[0]:
            analysis_table = pd.concat([analysis_table, pd.DataFrame([[result[result['popularity'] == i]['horse_nationality'].values[0], 
                                                              result[result['popularity'] == i]['popularity'].values[0], 
                                                              result[result['popularity'] == i]['place'].values[0], vote['odds'].values[0], 
                                                              race_name]],
                                                              columns=['horse_nationality', 'popularity', 'place', 'rebate', 'race'])])
        else:
            analysis_table = pd.concat([analysis_table, pd.DataFrame([[result[result['popularity'] == i]['horse_nationality'].values[0], 
                                                              result[result['popularity'] == i]['popularity'].values[0], 
                                                              result[result['popularity'] == i]['place'].values[0], 
                                                              0, 
                                                              race_name]],
                                                              columns=['horse_nationality', 'popularity', 'place', 'rebate', 'race'])])


2016arc
2016melbourne
2016bc
2016hkir_vase
2016hkir_sprint
2016hkir_mile
2016hkir_cup
2017dubai_turf
2017dubai_sc
2017dubai_wc
2017qe2
2017belmont
2017arc
2017hkir_vase
2017hkir_sprint
2017hkir_mile
2017hkir_cup
2018dubai_gs
2018dubai_turf
2018dubai_sc
2018dubai_wc
2018qe2
2018jlm
2018arc
2018cc
2018melbourne
2018hkir_vase
2018hkir_sprint
2018hkir_mile
2018hkir_cup
2019dubai_gs
2019dubai_turf
2019dubai_sc
2019dubai_wc
2019dm
2019hkch_2019csp
2019hkch_2019qe2
2019kd
2019belmont
2019pow
2019kgqes
2019ins
2019ics
2019arc
2019cc
2019cox
2019melbourne
2019hkir_vase
2019hkir_sprint
2019hkir_mile
2019hkir_cup
2020qes
2020ecl
2020arc
2020hkir_sprint
2020hkir_mile
2020hkir_cup
2021dubai_gs
2021dubai_turf
2021dubai_sc
2021dubai_wc
2021hkch_2021csp
2021hkch_2021qe2
2021arc


FileNotFoundError: [Errno 2] No such file or directory: 'data/2021arc_tbody.csv'

In [5]:
analysis_table

,horse_nationality,popularity,place,rebate,race
0,JPN,1.0,14,0,2016arc
0,IRE,2.0,5,0,2016arc
0,IRE,3.0,1,780,2016arc
0,GB,1.0,3,0,2016melbourne
0,IRE,2.0,13,0,2016melbourne
...,...,...,...,...,...
0,AUS,2.0,1,710,2021hkch_2021csp
0,AUS,3.0,12,0,2021hkch_2021csp
0,JPN,1.0,3,0,2021hkch_2021qe2
0,JPN,2.0,2,0,2021hkch_2021qe2


In [6]:
analysis_table[analysis_table['horse_nationality'] == 'JPN'].sort_values('rebate')

,horse_nationality,popularity,place,rebate,race
0,JPN,1.0,14,0,2016arc
0,JPN,1.0,2,0,2018hkir_cup
0,JPN,2.0,4,0,2018hkir_cup
0,JPN,2.0,2,0,2021hkch_2021qe2
0,JPN,3.0,4,0,2019dubai_turf
0,JPN,1.0,6,0,2019dubai_sc
0,JPN,3.0,3,0,2019dubai_sc
0,JPN,2.0,3,0,2019hkch_2019qe2
0,JPN,2.0,5,0,2018hkir_mile
0,JPN,3.0,6,0,2019hkch_2019qe2


In [7]:
analysis_table[analysis_table['horse_nationality'] == 'JPN']['rebate'].value_counts()

0      40
330     2
550     2
470     1
280     1
160     1
170     1
120     1
Name: rebate, dtype: int64

In [8]:
print(analysis_table[analysis_table['horse_nationality'] == 'JPN'].shape)
print(analysis_table[analysis_table['horse_nationality'] == 'JPN']['rebate'].sum())


(49, 5)
2960


In [9]:
analysis_table[analysis_table['horse_nationality'] != 'JPN'].sort_values('rebate')

,horse_nationality,popularity,place,rebate,race
0,IRE,2.0,5,0,2016arc
0,GB,2.0,4,0,2019arc
0,GB,1.0,2,0,2019arc
0,GB,2.0,7,0,2019ics
0,IRE,2.0,4,0,2019ins
...,...,...,...,...,...
0,AUS,2.0,1,710,2021hkch_2021csp
0,GB,3.0,1,760,2019ins
0,USA,3.0,1,760,2018dubai_gs
0,IRE,3.0,1,780,2016arc


In [10]:
analysis_table[analysis_table['horse_nationality'] != 'JPN']['rebate'].value_counts()

0      109
760      2
180      2
320      2
170      2
120      2
260      2
160      1
230      1
200      1
710      1
450      1
690      1
420      1
140      1
150      1
390      1
870      1
580      1
310      1
560      1
300      1
540      1
280      1
780      1
250      1
Name: rebate, dtype: int64

In [11]:
print(analysis_table[analysis_table['horse_nationality'] != 'JPN'].shape)
print(analysis_table[analysis_table['horse_nationality'] != 'JPN']['rebate'].sum())


(140, 5)
11630


In [12]:
hk = analysis_table[analysis_table['race'].str.contains('hk')]


In [13]:
hk[hk['horse_nationality'] == 'JPN']['rebate'].value_counts()

0      24
550     2
160     1
330     1
470     1
Name: rebate, dtype: int64

In [14]:
print(hk[hk['horse_nationality'] == 'JPN'].shape)
print(hk[hk['horse_nationality'] == 'JPN']['rebate'].sum())


(29, 5)
2060


In [15]:
hk[hk['horse_nationality'] != 'JPN']['rebate'].value_counts()

0      31
710     1
200     1
140     1
160     1
420     1
690     1
180     1
250     1
320     1
Name: rebate, dtype: int64

In [16]:
print(hk[hk['horse_nationality'] != 'JPN'].shape)
print(hk[hk['horse_nationality'] != 'JPN']['rebate'].sum())


(40, 5)
3070


In [17]:
not_hk = analysis_table[~analysis_table['race'].str.contains('hk')]


In [18]:
not_hk[not_hk['horse_nationality'] == 'JPN']['rebate'].value_counts()

0      16
330     1
170     1
120     1
280     1
Name: rebate, dtype: int64

In [19]:
print(not_hk[not_hk['horse_nationality'] == 'JPN'].shape)
print(not_hk[not_hk['horse_nationality'] == 'JPN']['rebate'].sum())


(20, 5)
900


In [20]:
not_hk[not_hk['horse_nationality'] != 'JPN']['rebate'].value_counts()

0      78
170     2
760     2
260     2
120     2
280     1
540     1
150     1
300     1
780     1
180     1
310     1
320     1
450     1
580     1
390     1
870     1
230     1
560     1
Name: rebate, dtype: int64

In [21]:
print(not_hk[not_hk['horse_nationality'] != 'JPN'].shape)
print(not_hk[not_hk['horse_nationality'] != 'JPN']['rebate'].sum())


(100, 5)
8560


In [22]:
vote

,no,odds,popularity
0,7,550,3


In [23]:
result['horse_nationality'] = result['horse'].apply(lambda x: x[x.find('(')+1: len(x)-1])

In [24]:
for i in range(1,4):
    if result[result['popularity'] == i]['no'].values[0] == vote['no'].values[0]:
        analysis_table = pd.concat([analysis_table, pd.DataFrame([[result[result['popularity'] == i]['horse_nationality'].values[0], 
                                                                  result[result['popularity'] == i]['popularity'].values[0], 
                                                                  result[result['popularity'] == i]['place'].values[0], vote['odds'].values[0]]], 
                                                                 columns=['horse_nationality', 'popularity', 'place', 'rebate'])])
    else:
        analysis_table = pd.concat([analysis_table, pd.DataFrame([[result[result['popularity'] == i]['horse_nationality'].values[0], 
                                                                  result[result['popularity'] == i]['popularity'].values[0], 
                                                                  result[result['popularity'] == i]['place'].values[0], 0]], 
                                                                 columns=['horse_nationality', 'popularity', 'place', 'rebate'])])
    

In [25]:
'10(12)'.find('(')

2

In [26]:
result

,place,no,horse,gender_age,weight,jockey,time,horse_weight,trainer,popularity,horse_nationality
0,1,7,ラヴズオンリーユー(JPN),牝5,55.5,C.ホー,2:01.22,478,矢作 芳人(JPN),3,JPN
1,2,1,グローリーヴェイズ(JPN),牡6,57.0,K.ティータン,3/4,452,尾関 知人(JPN),2,JPN
2,3,6,デアリングタクト(JPN),牝4,55.5,松山 弘平,1 1/4,464,杉山 晴紀(JPN),1,JPN
3,4,4,キセキ(JPN),牡7,57.0,C.スコフィールド,2 3/4,504,辻野 泰之(JPN),5,JPN
4,5,2,エグザルタント(IRE),せん7,57.0,Z.パートン,3,480,A.クルーズ(HK),4,IRE
5,6,3,グロリアスドラゴン(IRE),せん6,57.0,M.プーン,3 3/4,524,K.ルイ(HK),6,IRE
6,7,5,タイムワープ(GB),せん8,57.0,J.モレイラ,5 1/2,566,A.クルーズ(HK),7,GB


In [27]:
result.dropna(subset=['popularity'])

,place,no,horse,gender_age,weight,jockey,time,horse_weight,trainer,popularity,horse_nationality
0,1,7,ラヴズオンリーユー(JPN),牝5,55.5,C.ホー,2:01.22,478,矢作 芳人(JPN),3,JPN
1,2,1,グローリーヴェイズ(JPN),牡6,57.0,K.ティータン,3/4,452,尾関 知人(JPN),2,JPN
2,3,6,デアリングタクト(JPN),牝4,55.5,松山 弘平,1 1/4,464,杉山 晴紀(JPN),1,JPN
3,4,4,キセキ(JPN),牡7,57.0,C.スコフィールド,2 3/4,504,辻野 泰之(JPN),5,JPN
4,5,2,エグザルタント(IRE),せん7,57.0,Z.パートン,3,480,A.クルーズ(HK),4,IRE
5,6,3,グロリアスドラゴン(IRE),せん6,57.0,M.プーン,3 3/4,524,K.ルイ(HK),6,IRE
6,7,5,タイムワープ(GB),せん8,57.0,J.モレイラ,5 1/2,566,A.クルーズ(HK),7,GB


In [28]:
result['no']

0    7
1    1
2    6
3    4
4    2
5    3
6    5
Name: no, dtype: int64

In [29]:
vote

,no,odds,popularity
0,7,550,3
